In [ ]:
checkpoint = widgets.Dropdown(options=[('свежая', 1), ('стабильная', 2)], value=1, description='версия A1111:').add_class("checkpoint")
gdrive = widgets.Checkbox(value=False, description='подключить гуглодиск').add_class("googledisk_mount")
gdrive_models = widgets.Checkbox(value=False, description='подтянуть модели/лоры/etc с гуглодиска').add_class("googledisk_symlink")
gfolder = widgets.Text(value="/content/drive/MyDrive/SD/", placeholder='там где лежат папки с моделями и т.п.').add_class("gfolder")
gdrive_only = widgets.Checkbox(value=False, description='модели только с гуглодиска').add_class("googledisk_only")
xformers = widgets.Checkbox(value=True, description='xformers').add_class("xformers")
pix2pix = widgets.Checkbox(value=False, description='instruct pix2pix').add_class("pix2pix")
controlnet = widgets.Checkbox(value=False, description='ControlNet').add_class("controlnet")
extentions = widgets.Checkbox(value=False, description='дополнения').add_class("extentions")
extentions_mix = widgets.Checkbox(value=False, description='дополнения для слияние моделей').add_class("extentions_mix")
booru = widgets.Checkbox(value=False, description='DeepDanbooru').add_class("booru")
no_half_vae = widgets.Checkbox(value=True, description='--no-half-vae').add_class("no_half_vae")
starton = widgets.Checkbox(value=True, description='запустить сразу').add_class("starton")
gd_set = widgets.HBox([gdrive, gdrive_models, gfolder, gdrive_only]).add_class("gd_set")
eng = widgets.Checkbox(value=False, description='англоязычный интерфейс').add_class("eng")
ngrok_token = widgets.Text(value="", placeholder='тут можно указать свой токен ngrok').add_class("ngrok_token")
auth_token = widgets.HTML(value='''<a class="auth_token" title="получить свой токен Ngrok" href="https://dashboard.ngrok.com/get-started/your-authtoken" target="_blank">?</a>''')
# необязательный контент
additional_options = widgets.Accordion().add_class("additional_options")
additional_options.set_title(0, 'дополнительные опции')
additional_options.selected_index = None
loras = widgets.Checkbox(value=False, description='Gachi-LoRa').add_class("loras")
embeddings = widgets.Checkbox(value=False, description='внедрения').add_class("embeddings")
hypernet = widgets.Checkbox(value=False, description='гиперсети').add_class("hypernet")
additional_options.children = [widgets.HBox([loras, embeddings, hypernet]).add_class("extra_options")]

additions = widgets.HBox([controlnet, pix2pix, extentions, extentions_mix, booru, additional_options]).add_class("additions")
webuiargs = widgets.HBox([xformers, no_half_vae, starton, eng, ngrok_token, auth_token]).add_class("args")
apply = widgets.Button(description='продолжить ❯').add_class("apply")
model_header = HTML('<div><h2 id="model_header">выбор моделей</h2><p id="model_description"><a href="https://pr0lapse.github.io/models.html" target="_blank">описания моделей, ключи для их вызова</p></div>')
settings_first_header = HTML('<h2 id="setup_header">настройки</h2>')
def lora_to_extention():
  lpath = f"{fpath}models/Lora"
  extpath = f"{fpath}extensions/sd-webui-additional-networks/models/lora"
  extpath_dev2 = f"{fpath}extensions/sd-webui-additional-networks-dev2/models/lora"
  if not os.path.exists(lpath):
    os.makedirs(lpath)
  if not os.path.exists(extpath):
    os.makedirs(extpath)
  if not os.path.exists(extpath_dev2):
    os.makedirs(extpath_dev2)
  try:
    for l in os.listdir(lpath):
      try:
        os.symlink(f"{lpath}/{l}", f"{extpath}/{l}")
      except FileExistsError:
        pass
    for l in os.listdir(lpath):
      try:
        os.symlink(f"{lpath}/{l}", f"{extpath_dev2}/{l}")
      except FileExistsError:
        pass
  except:
    pass

#показать/спрятать указание папки на гдиске с моделями
def gd_path(change):
  if gdrive_models.value == True:
    gfolder.layout.display = 'flex'
    gdrive_only.layout.display = 'flex'
  if gdrive_models.value == False:
    gfolder.layout.display = 'none'
    gdrive_only.layout.display = 'none'

# примонтирование диска и показать/спрятать опции гдиска
def gdrive_mount(change):
  if gdrive.value == True:
    gdrive_models.layout.display = 'flex'
    gdrive_only.layout.display = 'flex'
    if not os.path.exists("/content/drive/MyDrive/"):
      try:
        drive.mount('/content/drive', force_remount=True)
        print('гуглодиск примонтирован')
      except:
        print("пользователь отказался")
  if gdrive.value == False:
    print("гуглодиск не примонтирован")
    gdrive_models.layout.display = 'none'
    gdrive_only.layout.display = 'none'
    gfolder.layout.display = 'none'
    gdrive_models.value = False
    gfolder.value = ""
    gdrive_only.value = False

#показать/спрятать виджет моделей (если модели только с гдиска - виджет не нужен)
def only_gdrive(b):
  if gdrive_only.value == True:
    tab.layout.display = 'none'
    search_and_button.layout.display = 'none'
    display(HTML('<style>#model_header,#model_description{display:none}'))
  if gdrive_only.value == False:
    tab.layout.display = 'flex'
    search_and_button.layout.display = 'flex'
    display(HTML('<style>#model_header,#model_description{display:block}'))

# установка вебуи и основных зависимостей, правки
# функция установки
def A1111_webui_install():
  try:
    Black = "stable-";Lives = "diffusion";Matter = "-webui";BlackLivesMatter = Black+Lives+Matter
    subprocess.run(["git", "clone", f"https://github.com/AUTOMATIC1111/{BlackLivesMatter}", fpath])
    %cd {fpath}
    if checkpoint_value == "new":
      %cd {fpath}
      !git checkout $commit_new
    if checkpoint_value == "legacy":
      %cd {fpath}
      !git checkout $commit_stable
    # переименование файлов
    os.rename(f'{fpath}webui.py', f'{fpath}{wname}')
    os.rename(f'{fpath}launch.py', f'{fpath}{lname}')
    !ln -s {fpath}{lname} {fpath}launch.py > /dev/null 2>&1
    !mkdir {fpath}huge_rectal_prolapse_in_google_colab > /dev/null 2>&1
    # установка репозиториев для вебуи
    with open(f'{fpath}{lname}', 'r') as fp:
      lines = fp.readlines()
      for line in lines:
        if line.find('STABLE_DIFFUSION_COMMIT_HASH') != -1:
          stable_diffusion_commit_hash = line.split('"')[1]
        if line.find('TAMING_TRANSFORMERS_COMMIT_HASH') != -1:
          taming_transformers_commit_hash = line.split('"')[1]
        if line.find('K_DIFFUSION_COMMIT_HASH') != -1:
          k_diffusion_commit_hash = line.split('"')[1]
        if line.find('CODEFORMER_COMMIT_HASH') != -1:
          codeformer_commit_hash = line.split('"')[1]
        if line.find('BLIP_COMMIT_HASH') != -1:
          blip_commit_hash = line.split('"')[1]
    openclip = "/root/.cache/huggingface/hub/models--openai--clip-vit-large-patch14/snapshots/8d052a0f05efbaefbc9e8786ba291cfdf93e5bff/"
    os.makedirs(openclip, exist_ok=True)
    !$wget {deps}cache.tar
    !tar -xvf cache.tar -C /
    !rm cache.tar
    reps = [
        f"git clone https://github.com/Stability-AI/stablediffusion.git {fpath}repositories/stable-diffusion-stability-ai && cd {fpath}repositories/stable-diffusion-stability-ai && git checkout {stable_diffusion_commit_hash}",
        f"git clone https://github.com/CompVis/taming-transformers.git {fpath}repositories/taming-transformers && cd {fpath}repositories/taming-transformers && git checkout {taming_transformers_commit_hash}",
        f"git clone https://github.com/crowsonkb/k-diffusion.git {fpath}repositories/k-diffusion && cd {fpath}repositories/k-diffusion && git checkout {k_diffusion_commit_hash}",
        f"git clone https://github.com/sczhou/CodeFormer.git {fpath}repositories/CodeFormer && cd {fpath}repositories/CodeFormer && git checkout {codeformer_commit_hash}",
        f"git clone https://github.com/salesforce/BLIP.git {fpath}repositories/BLIP && cd {fpath}repositories/BLIP && git checkout {blip_commit_hash}",
        f"git clone https://github.com/isl-org/MiDaS.git {fpath}repositories/midas",
        "wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/bin/cloudflared  && chmod +x /usr/bin/cloudflared",
        f"wget https://huggingface.co/utnah/clip/resolve/main/pytorch_model.bin -P {openclip}",
        ]
    def reps_inst(rep):
      !$rep
    with ThreadPoolExecutor(max_workers=len(reps)) as executor:
      futures = [executor.submit(reps_inst, rep) for rep in reps]
      for future in as_completed(futures):
          future.result()
    os.environ["SAFETENSORS_FAST_GPU"]='1'
    #print("подождите, происходит установка зависимостей...")
    if checkpoint_value == "legacy":
      print("подождите, происходит установка зависимостей...")
      !pip uninstall -y torchaudio torchvision torchtext xformers triton  > /dev/null 2>&1
      !python {lname} --share --xformers --ngrok test --no-download-sd-model --exit
    %cd {fpath}
    #!COMMANDLINE_ARGS="--exit" REQS_FILE="requirements_versions.txt" python {lname}
    #!COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python {lname}
    #!python f"{fpath}{lname}" --share --xformers --ngrok test --no-download-sd-model --ui-debug-mode --exit
    #!rm {fpath}modules/ui_tempdir.py
    #!$dl_gh'ui-tempdir.py' -O f"{fpath}modules/ui_tempdir.py"
    #!git clone https://github.com/Stability-AI/stablediffusion
    #!rm -rf /content/stablediffusion/.git /content/stablediffusion/.gitignore /content/stablediffusion/LICENSE /content/stablediffusion/LICENSE-MODEL /content/stablediffusion/README.md /content/stablediffusion/modelcard.md /content/stablediffusion/setup.py /content/stablediffusion/requirements.txt /content/sample_data 
    #distutils.dir_util.copy_tree('/content/stablediffusion', '/content/stable-diffusion-webui')
    #!rm -rf /content/stablediffusion
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {fpath}{wname}
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {fpath}modules/sd_models.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" {fpath}modules/extras.py
    !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' {fpath}modules/shared.py
    with open(f"{fpath}repositories/stable-diffusion-stability-ai/ldm/util.py") as r: text = r.read().replace('return get_obj_from_str(config["target"])(**config.get("params", dict()))', 'return get_obj_from_str(config["target"])(**config.get("params", dict())).cuda()')
    with open(f"{fpath}repositories/stable-diffusion-stability-ai/ldm/util.py", "w") as w: w.write(text)
    for root, dirs, files in os.walk(fpath):
      for file in files:
        if file.endswith('here.txt'):
          os.remove(os.path.join(root, file))
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции A1111_webui_install и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      error_message = str(traceback.format_exc())
      e = "в функции A1111_webui_install и получил "+str(e)
      
      print("Ошибка: ", error_message)

# скачивание модели и дополнения instruct-pix2pix
def instruct_pix2pix():
  try:
    if pix2pix.value == True:
      instructs = [
          f"git clone https://github.com/Klace/stable-diffusion-webui-instruct-pix2pix {fpath}extensions/stable-diffusion-webui-instruct-pix2pix",
          f"{dl} {sf}instruct-pix2pix.safetensors -c -P {fpath}models/Stable-diffusion/",
        ]
      def pix2pix_ext(instruct):
          !$instruct
      with ThreadPoolExecutor(max_workers=len(instructs)) as executor:
          futures = [executor.submit(pix2pix_ext, instruct) for instruct in instructs]
          for future in as_completed(futures):
              future.result()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции instruct_pix2pix и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции instruct_pix2pix и получил "+str(e)
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)
  if pix2pix.value == False:
    None
  clear()

# скачивание моделей и дополнения ControlNet
def ControlNet_install():
  try:
    if controlnet.value == True:
      !git clone https://github.com/Mikubill/sd-webui-controlnet {fpath}extensions/sd-webui-controlnet
      with open(f"{fpath}extensions/sd-webui-controlnet/scripts/controlnet.py") as r: text = r.read().replace('# "openpose_hand": openpose_hand,', '"openpose_hand": openpose_hand,')
      with open(f"{fpath}extensions/sd-webui-controlnet/scripts/controlnet.py", "w") as w: w.write(text)
      os.remove(f'{fpath}extensions/sd-webui-controlnet/models/cldm_v15.yaml')
      dlcn = f"wget -nv -t 10 --show-progress --progress=bar -q --content-disposition -P {fpath}extensions/sd-webui-controlnet/models "
      cnurl = sf+"controlnet/"
      controlnets = [
          "pip install svglib > /dev/null 2>&1",
          f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/cldm_v15.yaml",
          f"{dlcn} {cnurl}control_canny.safetensors",
          f"{dlcn} {cnurl}control_depth.safetensors",
          f"{dlcn} {cnurl}control_hed.safetensors",
          f"{dlcn} {cnurl}control_mlsd.safetensors",
          f"{dlcn} {cnurl}control_normal.safetensors",
          f"{dlcn} {cnurl}control_openpose.safetensors",
          f"{dlcn} {cnurl}control_scribble.safetensors",
          f"{dlcn} {cnurl}control_seg.safetensors",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/dpt_hybrid-midas-501f0c75.pt -P {fpath}extensions/sd-webui-controlnet/annotator/midas",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/mlsd_large_512_fp32.pth -P {fpath}extensions/sd-webui-controlnet/annotator/mlsd",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/mlsd_tiny_512_fp32.pth -P {fpath}extensions/sd-webui-controlnet/annotator/mlsd",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/upernet_global_small.pth -P {fpath}extensions/sd-webui-controlnet/annotator/uniformer",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/hand_pose_model.pth -P {fpath}extensions/sd-webui-controlnet/annotator/openpose",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/body_pose_model.pth -P {fpath}extensions/sd-webui-controlnet/annotator/openpose",
          f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/network-bsds500.pth -P {fpath}extensions/sd-webui-controlnet/annotator/hed",
          f"git clone https://github.com/fkunn1326/openpose-editor {fpath}extensions/openpose-editor"
        ]
      def controlnet_ext(controlnet):
          !$controlnet
      with ThreadPoolExecutor(max_workers=len(controlnets)) as executor:
          futures = [executor.submit(controlnet_ext, controlnet) for controlnet in controlnets]
          for future in as_completed(futures):
              future.result()
      import shutil
      controlnet_models_path = f"{fpath}extensions/sd-webui-controlnet/models/"
      cn_cofigs = ["control_sd15_canny.yaml", "control_sd15_depth.yaml", "control_sd15_hed.yaml", "control_sd15_mlsd.yaml", "control_sd15_normal.yaml", "control_sd15_openpose.yaml", "control_sd15_scribble.yaml", "control_sd15_seg.yaml"]
      for cn_cofig in cn_cofigs:
        shutil.copyfile(os.path.join(controlnet_models_path, cn_cofig), os.path.join(controlnet_models_path, cn_cofig.replace("_sd15", "")))
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции ControlNet_install и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      error_message = str(traceback.format_exc())
      e = "в функции ControlNet_install и получил "+str(e)
      
      print("Ошибка: ", error_message)
  if controlnet.value == False:
    None

# дополнение clip-interrogator
def interrogate_clip_install():
  try:
    !git clone --depth 1 https://github.com/pharmapsychotic/clip-interrogator-ext {fpath}extensions/clip-interrogator-ext
    ipath = f"{fpath}models/clip-interrogator"
    dl = f"{wget} -P {ipath} "
    clips = [
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_artists.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_flavors.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_mediums.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_movements.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_negative.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_trendings.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_artists.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_flavors.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_mediums.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_movements.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_negative.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_trendings.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_artists.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_flavors.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_mediums.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_movements.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_negative.safetensors",
        f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_trendings.safetensors",
        "pip install clip-interrogator==0.5.4 > /dev/null 2>&1",
        ]
    def clips_inst(clip):
      !$clip
    with ThreadPoolExecutor(max_workers=len(clips)) as executor:
      futures = [executor.submit(clips_inst, clip) for clip in clips]
      for future in as_completed(futures):
          future.result()
      clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции interrogate_clip_install и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции interrogate_clip_install и получил "+str(e)
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)

# обязательные дополнения
#"wget -q https://github.com/kohya-ss/sd-webui-additional-networks/archive/refs/heads/dev2.zip",
#f"git clone https://github.com/freecoderai01/custom-hires-fix {fpath}extensions/custom-hires-fix",
#f"git clone https://github.com/hnmr293/sd-webui-llul {fpath}extensions/sd-webui-llul",
#f"git clone https://github.com/richrobber2/canvas-zoom {fpath}extensions/canvas-zoom",
def extentions_builtin():
  try:
    #таймер, описания моделей и апскейлер, замена лиц, пропорции картинки, генератор промптов
    os.makedirs(f'{fpath}static', exist_ok = True)
    builtins = [
      
      f"git clone https://github.com/xxxOVALxxx/web-ui-fix/ {fpath}extensions-builtin/fix-webui",
      f"git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 {fpath}extensions/ultimate-upscale-for-automatic1111",
      f"git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon {fpath}extensions/a1111-sd-webui-locon",
      f"git clone https://github.com/kohya-ss/sd-webui-additional-networks {fpath}extensions/sd-webui-additional-networks",
      f"git clone https://github.com/adieyal/sd-dynamic-prompts {fpath}extensions/sd-dynamic-prompts",
      f"git clone https://github.com/yownas/shift-attention {fpath}extensions/shift-attention",
      f"git clone https://github.com/yownas/seed_travel {fpath}extensions/seed_travel",
      f"git clone https://github.com/Filarius/video2video {fpath}extensions/video2video",
      f"git clone https://github.com/kex0/batch-face-swap {fpath}extensions/batch-face-swap",
      "pip install mediapipe dynamicprompts > /dev/null 2>&1",
      f"git clone https://github.com/thomasasfk/sd-webui-aspect-ratio-helper {fpath}extensions/sd-webui-aspect-ratio-helper",
    ]
    def ext_builtin_inst(builtin):
      !$builtin
    with ThreadPoolExecutor(max_workers=len(builtins)) as executor:
      futures = [executor.submit(ext_builtin_inst, builtin) for builtin in builtins]
      for future in as_completed(futures):
          future.result()
    #!unzip -n -q dev2.zip -d {fpath}extensions && rm dev2.zip
    !echo -n {launching} > {fpath}static/launch.txt
    %cd {fpath}extensions/a1111-sd-webui-locon
    !git checkout 04b768bad41e0f1122a86d0636fce47bb2765af8
    %cd {fpath}
    #скрипты
    scrurls = [
        "https://github.com/feffy380/prompt-morph/raw/master/prompt_morph.py",
        "https://github.com/Animator-Anon/Animator/raw/main/animation.py",
        "https://github.com/Animator-Anon/Animator/raw/main/animation_v6.py",
        "https://github.com/memes-forever/Stable-diffusion-webui-video/raw/main/videos.py",
        "https://github.com/dfaker/embedding-to-png-script/raw/main/embedding_to_png.py",
        "https://github.com/dfaker/stable-diffusion-webui-cv2-external-masking-script/raw/main/external_masking.py",
    ]
    def download_sdscripts(scrurl):
      !$dl $scrurl -P {fpath}scripts
    with ThreadPoolExecutor(max_workers=len(scrurls)) as executor:
      futures = [executor.submit(download_sdscripts, scrurl) for scrurl in scrurls]
      for future in as_completed(futures):
          future.result()
    clear()
  except Exception as e:
    try:
      e = "в функции extentions_builtin и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции extentions_builtin и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)

#скачивание интеррогейтов и языковых моделей для составления промпта (быстрее чем через вебуи)
def misc_models():
  try:
    miscmodels = [
    f"{wget} https://github.com/AUTOMATIC1111/TorchDeepDanbooru/releases/download/v1/model-resnet_custom_v3.pt -P {fpath}models/torch_deepdanbooru",
    f"{wget} https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_caption_capfilt_large.pth -P {fpath}models/BLIP",
    f"wget -q {gh}artists.txt -P {fpath}interrogate",
    f"wget -q {gh}flavors.txt -P {fpath}interrogate",
    f"wget -q {gh}mediums.txt -P {fpath}interrogate",
    f"wget -q {gh}movements.txt -P {fpath}interrogate",
    ]
    def misc_models_inst(miscmodel):
      !$miscmodel
    with ThreadPoolExecutor(max_workers=len(miscmodels)) as executor:
      futures = [executor.submit(misc_models_inst, miscmodel) for miscmodel in miscmodels]
      for future in as_completed(futures):
          future.result()
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции misc_models и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции misc_models и получил "+str(e)
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)

# скачивание дополнений/скриптов
def extentions_install():
  # f"git clone https://github.com/dfaker/SD-latent-mirroring {fpath}extensions/latent-mirroring",
  try:
    if extentions.value == True:
      sdextentions = [
      f"git clone https://github.com/jtydhr88/sd-3dmodel-loader {fpath}extensions/sd-3dmodel-loader",
      f"git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-aesthetic-gradients {fpath}extensions/aesthetic-gradients",
      f"git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-wildcards {fpath}extensions/wildcards",
      f"git clone https://github.com/yfszzx/stable-diffusion-webui-inspiration {fpath}extensions/inspiration",
      f"git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-tokenizer {fpath}extensions/tokenizer",
      f"git clone https://github.com/OedoSoldier/enhanced-img2img {fpath}extensions/enhanced-img2img",
      ]
      def ext_sdextention_inst(sdextention):
        !$sdextention
      with ThreadPoolExecutor(max_workers=len(sdextentions)) as executor:
        futures = [executor.submit(ext_sdextention_inst, sdextention) for sdextention in sdextentions]
        for future in as_completed(futures):
            future.result()
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции extentions_install и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции extentions_install и получил "+str(e)
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)
  if extentions.value == False:
    None

def extentions_mix_install():
  try:
    if extentions_mix.value == True:
      mixextentions = [
      f"git clone https://github.com/hako-mikan/sd-webui-supermerger {fpath}extensions/sd-webui-supermerger",
      f"git clone https://github.com/arenatemp/stable-diffusion-webui-model-toolkit {fpath}extensions/stable-diffusion-webui-model-toolkit",
      f"git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui {fpath}extensions/sdweb-merge-block-weighted-gui",
      f"git clone https://github.com/bbc-mc/sdweb-merge-board {fpath}extensions/sdweb-merge-board",
      ]
      def mixextention_inst(mixextention):
        !$mixextention
      with ThreadPoolExecutor(max_workers=len(mixextentions)) as executor:
        futures = [executor.submit(mixextention_inst, mixextention) for mixextention in mixextentions]
        for future in as_completed(futures):
            future.result()
    clear()
  except Exception as e:
    try:
      e = "в функции extentions_mix_install и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции extentions_mix_install и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)
  if extentions_mix.value == False:
    None

def upscalers():
  try:
    upscmodels = [
    f"wget -q https://huggingface.co/utnah/LDSR/resolve/main/project.yaml -P {fpath}models/LDSR",
    f"wget -nv -t 10 --show-progress --progress=bar -q --content-disposition https://huggingface.co/utnah/LDSR/resolve/main/model.safetensors -P {fpath}models/LDSR",
    f"{dl} {esrgan}4x-UltraSharp.pth -P {fpath}models/ESRGAN",
    f"{dl} {esrgan}4x-AnimeSharp.pth -P {fpath}models/ESRGAN",
    f"{dl} {esrgan}4x_foolhardy_Remacri.pth -P {fpath}models/ESRGAN",
    f"{dl} {esrgan}4xESRGAN.pth -P {fpath}models/ESRGAN",
    f"{dl} {esrgan}4x_Nickelback_70000G.pth -P {fpath}models/ESRGAN",
    f"{dl} {esrgan}lollypop.pth -P {fpath}models/ESRGAN",
    f"{dl} {esrgan}1x_artifacts_jpg_20_40_alsa.pth -P {fpath}models/ESRGAN",
    f"{dl} {esrgan}RealESRGAN_x4plus_anime_6B.pth -P {fpath}models/RealESRGAN",
    f"{dl} {esrgan}16xPSNR.pth -P {fpath}models/ESRGAN",
    f"{dl} {esrgan}4x_NMKD-Siax_200k.pth -P {fpath}models/ESRGAN",
    ]
    def upscmodels_inst(upscmodel):
      !$upscmodel
    with ThreadPoolExecutor(max_workers=len(upscmodels)) as executor:
      futures = [executor.submit(upscmodels_inst, upscmodel) for upscmodel in upscmodels]
      for future in as_completed(futures):
          future.result()
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции upscalers и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции upscalers и получил "+str(e)
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)

def emdeddings_install():
  if embeddings.value == True:
    try:
      !$dl $pt'embeddings' -O e.zip
      embunzip = f'7z -bsp1 -bso0 -y -bt x e.zip -o"{fpath}embeddings"'
      !$embunzip 
      !rm e.zip
      clear()
    except Exception as e:
      try:
        e = "в функции emdeddings_install и получил "+str(e)
        error_message = str(cgitb.text(sys.exc_info()))
        
        print("Ошибка: ", e)
      except Exception as e:
        import traceback
        e = "в функции emdeddings_install и получил "+str(e)
        error_message = str(traceback.format_exc())
        
        print("Ошибка: ", error_message)
  if embeddings.value == False:
    None
def emdeddings_for_negative():
  try:
    !$dl $pt'neg_embeddings' -O n.zip
    nembunzip = f'7z -bsp1 -bso0 -y -bt x n.zip -o"{fpath}embeddings"'
    !$nembunzip
    !rm n.zip
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции emdeddings_for_negative и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции emdeddings_for_negative и получил "+str(e)
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)

# deepdanbooru и автозавершение тегов
def deepdanbooru():
  try:
    if booru_value == True:
      !git clone "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git" {fpath}extensions/tag-autocomplete
      if not os.path.exists(f'{fpath}tmp'):
        os.makedirs(f'{fpath}tmp')
      tagacp = f"{fpath}tmp/tagAutocompletePath.txt"
      !touch $tagacp
      with open(tagacp, 'a') as ta:
        ta.write("extensions/a1111-sd-webui-tagcomplete/tags")
      !$wget $pt'deepdanbooru-v3-20211112-sgd-e28.zip' -P /content && unzip /content/deepdanbooru-v3-20211112-sgd-e28.zip  -d {fpath}models/deepbooru/ && rm /content/deepdanbooru-v3-20211112-sgd-e28.zip
    if booru_value == False:
      None
  except Exception as e:
    try:
      e = "в функции deepdanbooru и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции deepdanbooru и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)
  clear()

# скачивание VAE
def VAE():
  try:
    vae_path = f"{fpath}models/VAE"
    dl = f"{wget} -P {vae_path} "
    if not os.path.exists(vae_path):
      os.makedirs(vae_path)
    vaemodels = [
    f"{dl} {vae}blessed-fix.ckpt",
    f"{dl} {vae}blessed.ckpt",
    f"{dl} {vae}blessed2.ckpt",
    f"{dl} {vae}kl-f8-anime.ckpt",
    f"{dl} {vae}kl-f8-anime2.ckpt",
    f"{dl} {vae}kl-f8-trinart_characters.ckpt",
    f"{dl} {vae}novelai-vae.ckpt",
    f"{dl} {vae}vae-ft-ema.ckpt",
    f"{dl} {vae}vae-ft-mse.ckpt",
    f"{dl} {vae}color.vae.safetensors",
    ]
    def vaemodels_inst(vaemodel):
      !$vaemodel
    with ThreadPoolExecutor(max_workers=len(vaemodels)) as executor:
      futures = [executor.submit(vaemodels_inst, vaemodel) for vaemodel in vaemodels]
      for future in as_completed(futures):
          future.result()
    clear()
  except Exception as e:
    try:
      e = "в функции VAE и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции VAE и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)

# скачивание гиперсетей
def hypernets_install():
  try:
    if hypernet.value == True:
      hpath = f"{fpath}models/hypernetworks"
      dl = f"{wget} -P {hpath} "
      hypernetworks = [
            f"{dl} '{hyper}LuisapAngels_v1.pt'",
            f"{dl} '{hyper}aki99-112000.pt'",
            f"{dl} '{hyper}LuisapOilPainting_v1.pt'",
            f"{dl} '{hyper}anime.pt'",
            f"{dl} '{hyper}anime_2.pt'",
            f"{dl} '{hyper}aini.pt'",
            f"{dl} '{hyper}henriRousseau_v1.pt'",
            f"{dl} '{hyper}gigantomastia_.pt'",
            f"{dl} '{hyper}photo-100k.pt'",
            f"{dl} '{hyper}curssHN slime.pt'",
            f"{dl} '{hyper}nullmono-23000.pt'",
            f"{dl} '{hyper}newhp6.pt'",
            f"{dl} '{hyper}bangyou-hyper-77500.pt'",
            f"{dl} '{hyper}watercolourv1_v10.pt'",
            f"{dl} '{hyper}LuisapMelted_v1.pt'",
            f"{dl} '{hyper}kyokucho-40000.pt'",
            f"{dl} '{hyper}mjv4Hypernetwork_v1.pt'",
            f"{dl} '{hyper}anime_3.pt'",
            f"{dl} '{hyper}molu-60000.pt'",
            f"{dl} '{hyper}spaceMarine_v10.pt'",
            f"{dl} '{hyper}mumumu-hyper-61000.pt'",
            f"{dl} '{hyper}bigrbear-10000.pt'",
            f"{dl} '{hyper}LuisapDarkage_pinguinstyle5Darkage.pt'",
            f"{dl} '{hyper}samdoesartsHypernetwork_v422500Steps.pt'",
            f"{dl} '{hyper}mechanicus_v10.pt'",
            f"{dl} '{hyper}phanTattoo_phantatt.pt'",
            f"{dl} '{hyper}j9063DStyle_.pt'",
          ]
      def hypernetworks_inst(hypernetwork):
        !$hypernetwork
      with ThreadPoolExecutor(max_workers=len(hypernetworks)) as executor:
        futures = [executor.submit(hypernetworks_inst, hypernetwork) for hypernetwork in hypernetworks]
        for future in as_completed(futures):
            future.result()
        clear()
    if hypernet.value == False:
      None
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции hypernets_install и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции hypernets_install и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)

# скачивание LORA
def LORA():
  try:
    if loras.value == True:
      if not os.path.exists(f'{fpath}models/Lora'):
        os.makedirs(f'{fpath}models/Lora')
      path = f"{fpath}models/Lora/"
      dl = f"{wget} -P {path} "
      loramodels = [
          f"{dl} {lora}Seiji_Shingen_idol-master.safetensors",
          f"{dl} {lora}PhotoVintage.pt",
          f"{dl} {lora}SilverjowStyle.safetensors",
          f"{dl} {lora}ArtisticJinskyStyle.safetensors",
          f"{dl} {lora}uncutPenis.safetensors",
          f"{dl} {lora}hoacb.safetensors",
          f"{dl} {lora}penguinFrontierStyle.safetensors",
          f"{dl} {lora}anime_penis.safetensors",
          f"{dl} {lora}priapusATarouMentaiko.safetensors",
          f"{dl} {lora}husbando.safetensors",
          f"{dl} {lora}jockstrap.safetensors",
          f"{dl} {lora}aenaluckStyle.safetensors",
          f"{dl} {lora}fConStyle.safetensors",
          f"{dl} {lora}wrestlingSinglets.safetensors",
          f"{dl} {lora}rnarccusStyle.safetensors",
          f"{dl} {lora}unionSuits.safetensors",
          f"{dl} {lora}pissDrinking.safetensors",
          f"{dl} {lora}StepanGilev.safetensors",
          f"{dl} {lora}Lozhkin.safetensors",
          f"{dl} {lora}AntonyStarr_10.safetensors",
          f"{dl} {lora}turbomale_LoraV6.safetensors",
          f"{dl} {lora}gigachadDiffusionLora_v69.safetensors",
          f"{dl} {lora}niceMenModel_menModelV10.safetensors",
          f"{dl} {lora}jacobFryeAssassins_v1.safetensors",
          f"{dl} {lora}sxzLeonSKennedyEduard_sxzLeon.safetensors",
          f"{dl} {lora}kensouTheKingOf_10.safetensors",
          f"{dl} {lora}pantsTugMemeTrend_v1.safetensors",
          f"{dl} {lora}ChiakiMorisawa_v10.safetensors",
          f"{dl} {lora}morphedMuscle_10b.safetensors",
          f"{dl} {lora}husbandoLocon_v31.safetensors",
          f"{dl} {lora}uncutPenisLora_assholeInpanting.safetensors",
          f"{dl} {lora}tekkowangStyle_v1.safetensors",
          f"{dl} {lora}DukeNukem_v1.safetensors",
          f"{dl} {lora}Penis_anime_lora.safetensors",
          f"{dl} {lora}flaccidPenisAnimeLora_v2.safetensors",
          f"{dl} {lora}airoticartsPenisModel_v10Lora.safetensors",
          f"{dl} {lora}onlycocks_v01.safetensors",
          f"{dl} {lora}bettercocks_v10.safetensors",
          f"{dl} {lora}bettercocks_v1Flaccid.safetensors",
        ]
      def dwnld_lora(loramodel):
          !$loramodel
      with ThreadPoolExecutor(max_workers=len(loramodels)) as executor:
          futures = [executor.submit(dwnld_lora, loramodel) for loramodel in loramodels]
          for future in as_completed(futures):
              future.result()
      clear()
    else:
      None
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции LORA и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции LORA и получил "+str(e)
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)

# дефолтные конфиги
def configs():
  try:
    ng() #нгрок
    defconfigs = [
        f"{dl_gh}localization.json -O {fpath}localizations/localization.json",
        f"{dl_gh}config_noGD.json -O {fpath}config.json",
        f"{dl_gh}ui-config.json -O {fpath}ui-config.json",
      ]
    def defaultgonf(defconfig):
        !$defconfig
    with ThreadPoolExecutor(max_workers=len(defconfigs)) as executor:
        futures = [executor.submit(defaultgonf, defconfig) for defconfig in defconfigs]
        for future in as_completed(futures):
            future.result()
    # сохранение картинок на гулодиск 
    if gdrive.value == True and os.path.exists("/content/drive/MyDrive"):
      with open(f"{fpath}config.json") as r: text = r.read().replace('/content/output/', '/content/drive/MyDrive/SD/')
      with open(f"{fpath}config.json", "w") as w: w.write(text)
      with open(f"{fpath}config.json") as r: text = r.read().replace('"control_net_allow_script_control": true', '"control_net_allow_script_control": true,\n    "def_batch_dir": "/content/drive/MyDrive/SD/ip2p-batch",\n    "outdir_ip2p_samples": "/content/drive/MyDrive/SD/ip2p-images"')
      with open(f"{fpath}config.json", "w") as w: w.write(text)
      if os.path.exists("/content/drive/MyDrive/SD/styles.csv"):
        !rm {fpath}styles.csv {fpath}styles.csv.bak
        !ln -s /content/drive/MyDrive/SD/styles.csv {fpath}styles.csv
        !ln -s /content/drive/MyDrive/SD/styles.csv {fpath}styles.csv.bak
        !chmod -c 777 {fpath}styles.csv
        !chmod -c 777 {fpath}styles.csv.bak
      else:
        !rm {fpath}styles.csv {fpath}styles.bak
        !$dl_gh'styles.csv' -O /content/drive/MyDrive/SD/styles.csv 
        !ln -s /content/drive/MyDrive/SD/styles.csv {fpath}styles.csv
        !ln -s /content/drive/MyDrive/SD/styles.csv {fpath}styles.csv.bak
        !chmod -c 777 {fpath}styles.csv
        !chmod -c 777 {fpath}styles.csv.bak
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции configs и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции configs и получил "+str(e)
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)
  if gdrive.value == False or (not os.path.exists("/content/drive/MyDrive")):
    None

# англоязычный интерфейс
def eng_lang():
  try:
    if eng.value == True:
      !rm {fpath}localizations/localization.json
      with open(f"{fpath}config.json") as r: text = r.read().replace('"localization": "localization",', '"localization": "",')
      with open(f"{fpath}config.json", "w") as w: w.write(text)
    if eng.value == False:
      None
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции eng_lang и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции eng_lang и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)

# кастомизация
def customisation():
  try:
    !rm /usr/local/lib/python3.10/dist-packages/gradio/strings.py
    !$dl_gh'gradio_strings.py' -O /usr/local/lib/python3.10/dist-packages/gradio/strings.py
    !$dl_gh'notification.mp3' -O {fpath}notification.mp3
    !rm {fpath}html/footer.html
    !$dl_gh'footer.html' -O {fpath}html/footer.html
    with open(f"{fpath}modules/ui.py") as r: text = r.read().replace("print('ngrok authtoken detected, trying to connect...')", "")
    with open(f"{fpath}modules/ui.py", "w") as w: w.write(text)
    with open(f"{fpath}modules/sd_models.py") as r: text = r.read().replace('f"Checkpoint {model_checkpoint} not found; loading fallback {checkpoint_info.title}", file=sys.stderr', '"загрузка модели..."')
    with open(f"{fpath}modules/sd_models.py", "w") as w: w.write(text)
    with open("/usr/local/lib/python3.10/dist-packages/gradio/blocks.py") as r: text = r.read().replace('                strings.en["RUNNING_LOCALLY_SEPARATED"].format(\n                    self.protocol, self.server_name, self.server_port\n                )', '                  ""')
    with open("/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", "w") as w: w.write(text)
    with open("/usr/local/lib/python3.10/dist-packages/gradio/blocks.py") as r: text = r.read().replace('strings.en["SHARE_LINK_DISPLAY"].format(self.share_url)', '"\033[01;38;05;112m⯈\033[0m Ссылка на Градио:", self.share_url')
    with open("/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", "w") as w: w.write(text)
    with open(f"{fpath}modules/ngrok.py") as r: text = r.read().replace("ngrok connected to localhost:{port}! URL: {public_url}\\n'\n               'You can use this link after the launch is complete.", "\\033[01;38;05;112m⯈\\033[0m Ссылка на Нгрок: {public_url}")
    with open(f"{fpath}modules/ngrok.py", "w") as w: w.write(text)
    with open(f"{fpath}modules/ngrok.py") as r: text = r.read().replace("f'Invalid ngrok authtoken, ngrok connection aborted.\\n'\n              f'Your token: {token}, get the right one on https://dashboard.ngrok.com/get-started/your-authtoken'", "'указан неверный токен или тебе выпал общий токен который использует другой анон, получи свежий: https://dashboard.ngrok.com/get-started/your-authtoken'")
    with open(f"{fpath}modules/ngrok.py", "w") as w: w.write(text)
    with open(f'{fpath}javascript/ui.js', 'r') as uijs: uijs_content = uijs.read()
    with open(f'{fpath}javascript/ui.js', 'r') as uijs: uijs_content = uijs.read()
    warning_uijs = uijs_content.replace('function restart_reload(){\n    document.body.innerHTML=\'<h1 style="font-family:monospace;margin-top:20%;color:lightgray;text-align:center;">Reloading...</h1>\';\n    setTimeout(function(){location.reload()},2000)\n\n    return []\n}', 'function restart_reload(){\n    document.body.innerHTML=\'<h1 style="font-family:monospace;margin-top:20%;color:red;text-align:center;">Чтобы выполнить перезагрузку, нужно остановить выполнение кода в ячейке колаба:<br>Ctrl+M+I или "Среда выполнения -> Остановить выполнение кода"<br>Затем нужно выполнить перезапуск через ячейку перезапуска!<br>Штатная функция перезапуска ВебУИ не работает через колаб!!!<br>Останавливать и перезапускать нужно каждый раз после добавления/изменения дополнений, списка экстрасетей, скриптов, и т.п.</h1>\';\n    setTimeout(function(){location.reload()},40000)\n\n    return []\n}')
    with open(f'{fpath}javascript/ui.js', 'w') as uijs: uijs.write(warning_uijs)
    def remove_line(file_path, line_to_remove, new):
      try:
        with open(file_path) as r:
          text = r.read().replace(f"""{line_to_remove}""", f"""{new}""")
        with open(file_path, "w") as w: w.write(text)
      except:
        None
    os.environ["DISPLAY"] = "ХУЙ"
    !rm /root/.ngrok2/ngrok.yml > /dev/null 2>&1
    remove_line("/usr/local/lib/python3.10/dist-packages/pyngrok/conf.py", """DEFAULT_NGROK_CONFIG_PATH = os.path.join(os.path.expanduser("~"), ".ngrok2", "ngrok.yml")""", '''DEFAULT_NGROK_CONFIG_PATH = "/root/.config/ngrok/ngrok.yml"''')
    remove_line(f"{fpath}repositories/stable-diffusion-stability-ai/ldm/util.py", """if verbose:\n        print(f"{model.__class__.__name__} has {total_params*1.e-6:.2f} M params.")""", "")
    remove_line(f"{fpath}extensions-builtin/LDSR/sd_hijack_ddpm_v1.py", """print(f"{self.__class__.__name__}: Running in {self.parameterization}-prediction mode")""", "")
    remove_line(f"{fpath}modules/models/diffusion/ddpm_edit.py", """print(f"{self.__class__.__name__}: Running in {self.parameterization}-prediction mode")""", "")
    remove_line(f"{fpath}repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py", """print(f"{self.__class__.__name__}: Running in {self.parameterization}-prediction mode")""", "")
    remove_line(f"{fpath}{wname}", """print(f"Startup time: {startup_timer.summary()}.")""", "")
    remove_line(f"{fpath}{wname}", """print(f"Startup time: {startup_timer.summary()}.")""", "")
    remove_line(f"{fpath}{wname}", """print(f'Interrupted with signal {sig} in {frame}')""", "")
    remove_line(f"{fpath}modules/sd_models.py", """print(f"Model loaded in {timer.summary()}.")""", "")
    remove_line(f"{fpath}extensions/sd-webui-additional-networks/scripts/model_util.py", """print("[AddNet] Updating model hashes...")""", "")
    remove_line(f"{fpath}modules/textual_inversion/textual_inversion.py", """print(f"Textual inversion embeddings skipped({len(self.skipped_embeddings)}): {', '.join(self.skipped_embeddings.keys())}")""", "")
    remove_line(f"{fpath}modules/textual_inversion/textual_inversion.py", """print(f"Textual inversion embeddings loaded({len(self.word_embeddings)}): {', '.join(self.word_embeddings.keys())}")""", "")
    remove_line(f"{fpath}modules/textual_inversion/textual_inversion.py", """if len(self.skipped_embeddings) > 0:""", "")
    remove_line(f"{fpath}modules/sd_hijack.py", """print("Applying xformers cross attention optimization.")""", "")
    remove_line(f"{fpath}extensions/a1111-sd-webui-locon/scripts/main.py", """print('LoCon Extension hijack addnet extension successfully')""", "")
    remove_line(f"{fpath}extensions/a1111-sd-webui-locon/scripts/main.py", """print('Additional Network extension not installed, Only hijack built-in lora')""", """None""")
    remove_line(f"{fpath}extensions/a1111-sd-webui-locon/scripts/main.py", """print('LoCon Extension hijack built-in lora successfully')""", """None""")
    remove_line(f"{fpath}extensions/sd-webui-additional-networks/scripts/model_util.py", """thread_count = max(1, int(shared.opts.data.get("additional_networks_hash_thread_count", 1)))""", """thread_count = 4""")
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции customisation и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции customisation и получил "+str(e)
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)
  #clear()

# удалить пустые папки .ipynb_checkpoints и мусор
def cleanup():
  try:
    shutil.rmtree("/content/sample_data")
  except:
    None 
  sdwu = f"{fpath}"
  junk = ["webui.sh", "webui.bat", "webui-user.sh", "webui-user.bat", "webui-macos-env.sh", "screenshot.png", "README.md", "LICENSE.txt", "CODEOWNERS",]
  dirs_to_remove = glob.glob("/content/.ipynb_checkpoints", recursive=True)
  [os.remove(os.path.join(sdwu, file)) for file in junk if os.path.exists(os.path.join(sdwu, file))]
  for dir in dirs_to_remove:
    path = os.path.join(sdwu, dir)
    try:
      shutil.rmtree(path)
    except:
      None
  for root, dirs, files in os.walk(sdwu):
    for dir in dirs:
      if dir in ['.git', '.github', '.ipynb_checkpoints']:
        dir_path = os.path.join(root, dir)
        try:
          shutil.rmtree(dir_path)
        except OSError as e:
          None
  if os.path.exists(f"{fpath}models/Stable-diffusion/nullModel.ckpt"):
    os.remove(f"{fpath}models/Stable-diffusion/nullModel.ckpt")
  if os.path.exists("/root/.ngrok2/ngrok.yml"):
    try:
      shutil.move("/root/.ngrok2/ngrok.yml", "/root/.config/ngrok/ngrok.yml")
    except:
      None

def complite():
  try:
    clear()
    import psutil
    from subprocess import getoutput
    from IPython.display import HTML
    total_ram = round(psutil.virtual_memory().total / (1024**3), 2)
    free_ram = round(psutil.virtual_memory().available / (1024**3), 2)
    allocated_ram = round(total_ram - free_ram, 2)
    percent = round((free_ram / total_ram) * 100, 2)
    display(HTML("""<style>.setup_complite{color:var(--ansi-green);font-weight:700}hr.hr_setup_comlite{width:88%;position:relative;right:50%;border:none;border-top:2px var(--ansi-green) solid}.memory{color:var(--ansi-blue);font-weight:bold}.mem_allocate,.mem_all{color:var(--ansi-green)}.percent{color:var(--colab-primary-text-color)}.little_memory{display:flex;flex-wrap:wrap;width:50%;color:var(--ansi-red);font-weight:bold;font-family:sans-serif!important}.cell_fix{font-family:Consolas,monospace!important;color:var(--ansi-blue)}.cell_fix:before{content:"«";font-family:sans-serif!important;color:var(--ansi-blue)}.cell_fix:after{content:"»";font-family:sans-serif!important;color:var(--ansi-blue)}</style>"""))
    display(HTML(f'<p class="setup_complite">Установка завершена<hr class="hr_setup_comlite"/><span class="memory">Доступно ОЗУ: <span class="mem_allocate">{free_ram} Гб</span> из <span class="mem_all">{total_ram} Гб</span> <span class="percent">({percent}% свободно)</span></span></p>'))
    if int(free_ram) <= 7:
      display(HTML('<p class="little_memory">Осталось слишком мало памяти!<br/>Рекомендуется остановить ячейку и выполнить сброс памяти через ячейку <span class="cell_fix">фикс утечки памяти</span>, что находится выше</p>'))
    nsmi = getoutput('nvidia-smi')
    if "nvidia-smi: not found" in nsmi or "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running." in nsmi:
      display(HTML(f'{gh}warn'))
    else:
      try:
        card = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv').split('\n', 1)[1]
        vram = getoutput('nvidia-smi --query-gpu=memory.total --format=csv').split('\n', 1)[1]
        print("Видеокарта: \033[01;38;05;46mNvidia " + card + "\033[0m |" + "\033[01;38;05;188m Объём видеопамяти: \033[01;38;05;46m" + vram + "\033[0m ")
      except:
        print("nvidia-smi не ответил...")
    aloud("finish.wav")
    if starton_value == False:
      if starton_value == False:
        if xformers_value == True:
          xf = '--xformers'
        if xformers_value == False:
          xf = '--opt-split-attention'
        if no_half_vae_value == True:
          nhv = '--no-half-vae'
        if no_half_vae_value == False:
          nhv = ''
        gq = '--gradio-queue --disable-console-progressbars --no-hashing'
        ckpt_dir = f"{fpath}models/Stable-diffusion"
        if booru_value == True:
          deepdanbooru_arg = "--deepdanbooru"
        if booru_value == False:
          deepdanbooru_arg = ""
        default_args = f"--disable-safe-unpickle --enable-insecure-extension-access --share --listen {deepdanbooru_arg} --gradio-img2img-tool editor --theme dark --ckpt-dir {ckpt_dir}"
        ngrok_share = cfg+'cfg'
        freetoken = open(ngrok_share).read().split('\n')
        sharetoken = random.choice(list(freetoken))
        token = ngrok_token
        if not ngrok_token_value or len(ngrok_token_value) < 48 or len(ngrok_token_value) > 51:
          ngrok = f'--ngrok {sharetoken} --ngrok-region eu'
        else:
          ngrok = f'--ngrok {ngrok_token_value} --ngrok-region eu'
        cmdargs = f'{default_args} {gq} {xf} {nhv} {ngrok}'
        # сохранение переменных
        args_data = (xf, nhv, gq, deepdanbooru_arg, default_args)
        with open(f'{cfg}webui_args', 'wb') as webui_args:
          pickle.dump(args_data, webui_args)
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции complite и получил "+str(e)
      
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции complite и получил "+str(e)
      error_message = str(traceback.format_exc())
      
      print("Ошибка: ", error_message)

def run_webui():
  if starton_value == True:
    try:
      !sleep 3
      #Установка токена ngrok
      ngrok_share = cfg+'cfg'
      freetoken = open(ngrok_share).read().split('\n')
      sharetoken = random.choice(list(freetoken))
      token = ngrok_token
      if not ngrok_token_value or len(ngrok_token_value) < 48 or len(ngrok_token_value) > 51:
        ngrok = f'--ngrok {sharetoken} --ngrok-region eu'
      else:
        ngrok = f'--ngrok {ngrok_token_value} --ngrok-region eu'
      if xformers_value == True:
        xf = '--xformers'
      if xformers_value == False:
        xf = '--opt-split-attention'
      if no_half_vae_value == True:
        nhv = '--no-half-vae'
      if no_half_vae_value == False:
        nhv = ''
      gq = '--gradio-queue --disable-console-progressbars --no-hashing'
      ckpt_dir = f"{fpath}models/Stable-diffusion"
      if booru_value == True:
        deepdanbooru_arg = "--deepdanbooru"
      if booru_value == False:
        deepdanbooru_arg = ""
      default_args = f"--disable-safe-unpickle --enable-insecure-extension-access --share --listen {deepdanbooru_arg} --gradio-img2img-tool editor --theme dark --ckpt-dir {ckpt_dir}"
      cmdargs = f'{default_args} {gq} {xf} {nhv} {ngrok}'
      # сохранение переменных
      args_data = (xf, nhv, gq, deepdanbooru_arg, default_args)
      with open(f'{cfg}webui_args', 'wb') as webui_args:
        pickle.dump(args_data, webui_args)
      # запуск
      print("\033[1;36mВнимание! При проблемах с интерфейсом, используй другую ссылку, или перезагружай страницу, или выполни перезапуск.\033[0m \n")
      print("\033[1;34mПереходить по ссылкам можно после полной прогрузки, ожидай сообщения.\nСылки могут быть открыты автоматически, если в браузере выданы разрешения на переадресации и открытие окон для этого сайта.\033[0m")
      flara()
      print("")
      #Собственно запуск
      auto_opener()
      # в последней версии автоматик добавил аргументы: --no-hashing, можно использовать вместе с --lowram
      startwu = f"python {fpath}{wname} {cmdargs}"
      !rm /root/.ngrok2/ngrok.yml > /dev/null 2>&1
      !$startwu
    except Exception as e:
      try:
        error_message = str(cgitb.text(sys.exc_info()))
        e = "в функции run_webui и получил "+str(e)
        
        print("Ошибка: ", e)
      except Exception as e:
        e = "в функции run_webui и получил "+str(e)
        import traceback
        error_message = str(traceback.format_exc())
        
        print("Ошибка: ", error_message)
    if os.path.exists("/content/drive/MyDrive/SD/styles.csv"):
      !cp -f {fpath}styles.csv /content/drive/MyDrive/SD/styles.csv
    display(HTML('/content/.config/end'))
    if starton_value == False:
      print("тобой был отменен автозапуск после установки.\n033[01;38;05;141mчтобы выполнить запуск вебуи, нужно запустить ячейку «запуск/перезапуск (legacy)» или «запуск/перезапуск»\033[0m")

# фикс жора ОЗУ
def ram_liberator_jemalloc():
  !apt -y update -qq > /dev/null 2>&1
  !sudo apt-get install libjemalloc-dev > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbmalloc.so.2 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbmalloc_proxy.so.2 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbb.so.12 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbb.so.12 /usr/local/lib/libtbb.so.12 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbbind_2_5.so.3 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbbind.so.3 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbbind_2_0.so.3 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3 > /dev/null 2>&1

def ram_liberator():
  !$wget $gh_alt'libtcmalloc_minimal.so.4' -O '/lib/libtcmalloc_minimal.so.4' > /dev/null 2>&1
  while (os.path.getsize('/lib/libtcmalloc_minimal.so.4') < 300000) or (not os.path.exists('/lib/libtcmalloc_minimal.so.4')):
    !$wget $gh_alt'libtcmalloc_minimal.so.4' -O '/lib/libtcmalloc_minimal.so.4' > /dev/null 2>&1

# ver 19.02.2023
# lambda: on_download_click(None),
# interrogate_clip_install(),
def parallels_tasks():
    tasks = [
        extentions_builtin,
        extentions_install,
        extentions_mix_install,
        upscalers,
        emdeddings_install,
        emdeddings_for_negative,
        VAE,
        hypernets_install,
        deepdanbooru,
        instruct_pix2pix,
        ControlNet_install,
        misc_models,
    ]
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor: #временно 16 вместо (max_workers=len(tasks)) чтобы не задыхалось
        # шлем каждый task в отдельный executor
        futures = [executor.submit(run_task, task) for task in tasks]
        for future in concurrent.futures.as_completed(futures):
            # ждем завершения, выплевываем ошибки
            try:
                future.result()
            except Exception as e:
              try:
                error_message = str(cgitb.text(sys.exc_info()))
                e = "в функции parallels_tasks и получил "+str(e)
                
                print("Ошибка: ", e)
              except Exception as e:
                import traceback
                e = "в функции parallels_tasks и получил "+str(e)
                error_message = str(traceback.format_exc())
                
                print("Ошибка: ", error_message)

def run_task(task):
    # Новый ThreadPoolExecutor для каждого task
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # запускаем каждый task в новом executor
        executor.submit(task)

def store_values_external(): # тоже самое через внешний файл
  !$dl_gh_alt'store_values.ipynb' -P $cfg
  %run $cfg'store_values.ipynb'
  !rm $cfg'store_values.ipynb'

# еще один костыль: доустановка зависимостей после установки дополнений
def misc_dep_install():
  try:
    print("завершение установки...")
    %cd {fpath}
    loader(inprogress)
    !python {lname} --share --xformers --ngrok test --no-download-sd-model --exit  > /dev/null 2>&1
    subprocess.run(["pip", "install", "gradio==3.28.3"])
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции misc_dep_install и получил "+str(e)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции misc_dep_install и получил "+str(e)
      error_message = str(traceback.format_exc())
      print("Ошибка: ", error_message)

# все действия после нажатия на кнопку "продолжить"
#  mobile_player()
#  ram_liberator()
#  ultimate_gdrive_symlink()
def apply_button_click(b): # выполнение последрвательно всего выше (паузы  внутри самих def)
  try:
    print(new_id)
    try:
        clear()
        print("жди, установка уже началась...")
        store_values_external()
        depinst()
        A1111_webui_install()
        parallels_tasks()
        on_download_click(None)
        LORA()
        configs()
        eng_lang()
        misc_dep_install()
        #lora_to_extention()
        customisation()
        cleanup()
        complite()
        run_webui()
    except Exception as e:
      try:
        error_message = str(cgitb.text(sys.exc_info()))
        e = "в функции apply_button_click(b) "+str(e)

        print("Ошибка: ", e)
      except Exception as e:
        import traceback
        error_message = str(traceback.format_exc())
        e = "в функции apply_button_click(b) "+str(e)
        print("Ошибка: ", error_message)
  except:
    print("твоя копия блокнота устарела и не будет работать без ошибок, используй оригинальный блокнот:\nhttps://colab.research.google.com/drive/1TC4SSLncPWytSPvquR6Y4-U7wZRfAXrV")


